In [ ]:
import requests

In [ ]:
!pip install transformers datasets torch sentencepiece --use-deprecated=legacy-resolver

In [ ]:
pip install transformers==4.37.0 --use-deprecated=legacy-resolver

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
input_text = "My name is Manoj"
input_text_with_lang = f"eng_Latn {input_text}"

inputs = tokenizer(input_text_with_lang, return_tensors="pt")
outputs = model.generate(inputs['input_ids'], forced_bos_token_id=tokenizer.convert_tokens_to_ids("tel_Telu"))

translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(translated_text)

In [ ]:
import os
import re
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

langs_supported = {
    "eng_Latn": "en_XX",
    "tel_Telu": "te_IN",
}

def predict(batch, tokenizer, model, bos_token_id):
    encoded_batch = tokenizer(batch, padding=True, return_tensors="pt").to(model.device)
    generated_tokens = model.generate(
        **encoded_batch,
        num_beams=5,
        max_length=256,
        min_length=0,
        forced_bos_token_id=bos_token_id,
    )
    hypothesis = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return hypothesis

def main(txt_file, output_file, batch_size):
    enxx_model_name = "facebook/mbart-large-50-one-to-many-mmt"
    xxen_model_name = "facebook/mbart-large-50-many-to-one-mmt"
    tokenizers = {
        "enxx": AutoTokenizer.from_pretrained(enxx_model_name),
        "xxen": AutoTokenizer.from_pretrained(xxen_model_name),
    }
    models = {
        "enxx": AutoModelForSeq2SeqLM.from_pretrained(enxx_model_name).cuda(),
        "xxen": AutoModelForSeq2SeqLM.from_pretrained(xxen_model_name).cuda(),
    }

    for model_name in models:
        models[model_name].eval()

    with open(txt_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    english_sents = []
    telugu_sents = []
    for line in lines:
        if "++++$++++" in line:
            english, telugu = line.strip().split("++++$++++")
            english_sents.append(english)
            telugu_sents.append(telugu)
    english_sents = english_sents[:15000]
    telugu_sents = telugu_sents[:15000]
    translations = []

    print("Translating Telugu to English ...")
    tokenizers["xxen"].src_lang = langs_supported["tel_Telu"]
    bos_token_id = tokenizers["xxen"].lang_code_to_id[langs_supported["eng_Latn"]]

    telugu_to_english = []
    for i in tqdm(range(0, len(telugu_sents), batch_size)):
        start, end = i, int(min(len(telugu_sents), i + batch_size))
        batch = telugu_sents[start:end]
        telugu_to_english += predict(batch, tokenizers["xxen"], models["xxen"], bos_token_id)

    translations.append({"Telugu": telugu_sents, "English (Predicted)": telugu_to_english})

    print("Translating English to Telugu ...")
    tokenizers["enxx"].src_lang = langs_supported["eng_Latn"]
    bos_token_id = tokenizers["enxx"].lang_code_to_id[langs_supported["tel_Telu"]]

    english_to_telugu = []
    for i in tqdm(range(0, len(english_sents), batch_size)):
        start, end = i, int(min(len(english_sents), i + batch_size))
        batch = english_sents[start:end]
        english_to_telugu += predict(batch, tokenizers["enxx"], models["enxx"], bos_token_id)

    translations.append({"English": english_sents, "Telugu (Predicted)": english_to_telugu})

    with open(output_file, "w", encoding="utf-8") as f:
        f.write("Telugu (Original)++++$++++English (Predicted)\n")
        for t, e in zip(telugu_sents, telugu_to_english):
            f.write(f"{t}++++$++++{e}\n")
        f.write("\nEnglish (Original)++++$++++Telugu (Predicted)\n")
        for e, t in zip(english_sents, english_to_telugu):
            f.write(f"{e}++++$++++{t}\n")

    print(f"Translations saved to {output_file}")

txt_file = "english_telugu_data.txt"
output_file = "output_translations.txt"
batch_size = 8

if not torch.cuda.is_available():
    print("No GPU available")
    sys.exit(1)

main(txt_file, output_file, batch_size)


tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

...

Translating Telugu to English ...


100%|██████████| 1875/1875 [19:12<00:00,  1.63it/s]


Translating English to Telugu ...


100%|██████████| 1875/1875 [19:30<00:00,  1.60it/s]


Translations saved to output_translations.txt


In [ ]:
import os
from nltk.translate.meteor_score import meteor_score
import sacrebleu
import nltk
nltk.download('wordnet')
def tokenize_sentences(sentences):
    return [sentence.split() for sentence in sentences]

def calculate_meteor(references, hypotheses):
    references_tokenized = tokenize_sentences(references)
    hypotheses_tokenized = tokenize_sentences(hypotheses)
    scores = [meteor_score([ref], hyp) for ref, hyp in zip(references_tokenized, hypotheses_tokenized)]
    return sum(scores) / len(scores)

def calculate_bleu(references, hypotheses):
    bleu = sacrebleu.corpus_bleu(hypotheses, [references])
    return bleu.score

telugu_to_english = {}
english_to_telugu = {}


for l in open("english_telugu_data.txt", "r", encoding="utf-8").readlines():
    v = l.strip("\n").split("+$+")
    if len(v) < 2:
        continue
    telugu_to_english[v[1]] = v[0]
    english_to_telugu[v[0]] = v[1]


reference_tel_to_eng = []
hypothesis_tel_to_eng = []

reference_eng_to_tel = []
hypothesis_eng_to_tel = []


c = 0
with open("output_translations.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

reading_tel_to_eng = True
for l in lines[1:]:
    if l.strip() == "":
        reading_tel_to_eng = False
        continue

    v = l.strip("\n").split("++++$++++")
    if len(v) < 2:
        continue

    if reading_tel_to_eng:
        reference_tel_to_eng.append(telugu_to_english[v[0]])
        hypothesis_tel_to_eng.append(v[1])

    else:
        if v[0] == 'English (Original)': continue
        reference_eng_to_tel.append(english_to_telugu[v[0]])
        hypothesis_eng_to_tel.append(v[1])

meteor_tel_to_eng = calculate_meteor(reference_tel_to_eng, hypothesis_tel_to_eng)
print(f"METEOR Score for Telugu-to-English: {meteor_tel_to_eng}")

meteor_eng_to_tel = calculate_meteor(reference_eng_to_tel, hypothesis_eng_to_tel)
print(f"METEOR Score for English-to-Telugu: {meteor_eng_to_tel}")

bleu_tel_to_eng = calculate_bleu(reference_tel_to_eng, hypothesis_tel_to_eng)
print(f"BLEU Score for Telugu-to-English: {bleu_tel_to_eng}")

bleu_eng_to_tel = calculate_bleu(reference_eng_to_tel, hypothesis_eng_to_tel)
print(f"BLEU Score for English-to-Telugu: {bleu_eng_to_tel}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


METEOR Score for Telugu-to-English: 0.5682823773703624
METEOR Score for English-to-Telugu: 0.31329964582287123
BLEU Score for Telugu-to-English: 40.29333301248098
BLEU Score for English-to-Telugu: 10.952219610871493


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('all-MPNet-base-v2')

def calculate_semantic_similarity(references, hypotheses):
    ref_embeddings = model.encode(references, show_progress_bar=True)
    hyp_embeddings = model.encode(hypotheses, show_progress_bar=True)

    similarities = [cosine_similarity([ref], [hyp])[0][0] for ref, hyp in zip(ref_embeddings, hyp_embeddings)]
    return similarities

print("Calculating semantic similarity for Telugu-to-English...")
semantic_similarities_tel_to_eng = calculate_semantic_similarity(reference_tel_to_eng, hypothesis_tel_to_eng)
average_similarity_tel_to_eng = sum(semantic_similarities_tel_to_eng) / len(semantic_similarities_tel_to_eng)
print(f"Average Semantic Similarity for Telugu-to-English: {average_similarity_tel_to_eng:.4f}")

print("Calculating semantic similarity for English-to-Telugu...")
semantic_similarities_eng_to_tel = calculate_semantic_similarity(reference_eng_to_tel, hypothesis_eng_to_tel)
average_similarity_eng_to_tel = sum(semantic_similarities_eng_to_tel) / len(semantic_similarities_eng_to_tel)
print(f"Average Semantic Similarity for English-to-Telugu: {average_similarity_eng_to_tel:.4f}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/469 [00:00<?, ?it/s]

Average Semantic Similarity for Telugu-to-English: 0.7770
Calculating semantic similarity for English-to-Telugu...


Batches:   0%|          | 0/469 [00:00<?, ?it/s]

Batches:   0%|          | 0/469 [00:00<?, ?it/s]

Average Semantic Similarity for English-to-Telugu: 0.9555


In [ ]:
from transformers import MBartForConditionalGeneration, Trainer, TrainingArguments

model = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50-many-to-many-mmt')

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=3,
    push_to_hub=False,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

trainer.train()


In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")

predictions = []
references = []

for sample in val_data:
    inputs = tokenizer(sample["source"], return_tensors="pt", truncation=True).input_ids
    outputs = model.generate(inputs, max_length=50)
    predictions.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
    references.append([sample["target"]])

results = metric.compute(predictions=predictions, references=references)
print(f"BLEU Score: {results['score']}")


## Code


In [ ]:
pip install transformers datasets sacrebleu --use-deprecated=legacy-resolver


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you'll have fsspec 2024.9.0 which is incompatible.


In [ ]:
import numpy as np

english = []
telugu = []
with open("english_telugu_data.txt") as f:
  for line in f.readlines():
    e,t = line.split("++++$++++")
    english.append(e.strip("\n"))
    telugu.append(t.strip("\n"))

In [ ]:
telugu[0]

'అతని కాళ్ళు పొడవుగా ఉన్నాయి.'

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

data = pd.DataFrame({"source":telugu,"target":english})
print(data.head())
data = data.sample(frac=0.1)
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

data['source'] = data['source'].apply(clean_text)
data['target'] = data['target'].apply(clean_text)

train, test = train_test_split(data, test_size=0.15, random_state=42)
train, val = train_test_split(train, test_size=0.15, random_state=42)

print(f"Train size: {len(train)}, Validation size: {len(val)}, Test size: {len(test)}")


                                              source  \
0                       అతని కాళ్ళు పొడవుగా ఉన్నాయి.   
1            టామ్ ఫ్రెంచ్ మాట్లాడటం ఎలా నేర్పించారు?   
2              నేను ప్రతి రోజు సముద్రంలో ఈత కొడతాను.   
3  టామ్ కొంచెం పాలు కొనడానికి ఇంటికి వెళ్ళేటప్పుడ...   
4                                పొగ గదిని నింపింది.   

                                              target  
0                                 His legs are long.  
1                Who taught Tom how to speak French?  
2                       I swim in the sea every day.  
3  Tom popped into the supermarket on his way hom...  
4                             Smoke filled the room.  
Train size: 11256, Validation size: 1987, Test size: 2337


In [ ]:
pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 51.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments, DataCollatorForSeq2Seq

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
tokenizer.src_lang = "en_XX"  # Telugu
tokenizer.tgt_lang = "te_IN"  # English

model = MBartForConditionalGeneration.from_pretrained(model_name)

train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)

def tokenize_function(examples):
    model_inputs = tokenizer(examples["source"], max_length=64, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target"], max_length=64, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_dataset.map(tokenize_function, batched=True)
val_tokenized = val_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./mbart_results",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_total_limit=2,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

trainer.save_model("./final_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,1.254000,0.200499
1000,0.186800,0.164221
1500,0.152600,0.147356
2000,0.104700,0.140297
2500,0.100600,0.134816
3000,0.084600,0.136269
3500,0.061000,0.134307


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Step,Training Loss,Validation Loss
500,1.254000,0.200499
1000,0.186800,0.164221
1500,0.152600,0.147356
2000,0.104700,0.140297
2500,0.100600,0.134816
3000,0.084600,0.136269
3500,0.061000,0.134307
4000,0.060400,0.133127


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


In [ ]:
test_dataset = Dataset.from_pandas(test)

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import Dataset
from sacrebleu import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from sentence_transformers import SentenceTransformer, util
from nltk.tokenize import word_tokenize
import torch
import nltk
nltk.download('punkt_tab')

model_dir = "./final_model"
model = MBartForConditionalGeneration.from_pretrained(model_dir)
tokenizer = MBart50TokenizerFast.from_pretrained(model_dir)
tokenizer.src_lang = "te_IN"
tokenizer.tgt_lang = "en_XX"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_translations(dataset, model, tokenizer, max_length=64):
    predictions = []
    references = dataset["target"]

    for source in dataset["source"]:
        inputs = tokenizer(source, return_tensors="pt", max_length=max_length, truncation=True).to(device)

        outputs = model.generate(**inputs, max_length=max_length, num_beams=4)
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(translated_text)

    return predictions, references

predictions, references = generate_translations(test_dataset, model, tokenizer)

In [ ]:
bleu = corpus_bleu(predictions, [references])
print(f"BLEU Score: {bleu.score:.2f}")

tokenized_predictions = [word_tokenize(pred) for pred in predictions]
tokenized_references = [word_tokenize(ref) for ref in references]


BLEU Score: 39.55


In [ ]:
import nltk
nltk.download('wordnet')

meteor = sum(
    [meteor_score([ref], pred) for ref, pred in zip(tokenized_references, tokenized_predictions)]
) / len(tokenized_references)
print(f"METEOR Score: {meteor:.4f}")

similarity_model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings_pred = similarity_model.encode(predictions, convert_to_tensor=True)
embeddings_ref = similarity_model.encode(references, convert_to_tensor=True)

cosine_similarities = util.cos_sim(embeddings_pred, embeddings_ref)

avg_similarity = torch.mean(torch.diag(cosine_similarities)).item()
print(f"Semantic Similarity: {avg_similarity:.4f}")

[nltk_data] Downloading package wordnet to /root/nltk_data...


METEOR Score: 0.6054


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Semantic Similarity: 0.7541


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Facebook NLLB - 200

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_source = test['source'].tolist()
translations = []
for sentence in test_source:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=128)
    translations.append(tokenizer.decode(outputs[0], skip_special_tokens=True))

test['predicted'] = translations
